<h1 align=center>PyLIE: Profiling</h1>
<h3 align=center>24 October 2017</h3>

## What is profiling?

Whether or not the profiler is *deterministic* or *statistical*, a profile is a set of statistics that describes how often and for how long various parts of the program executed.

Within the standard library is the `cProfile` and `Profile` modules that are native *deterministic* profilers. The key difference between the two are the code base implementations: C and Python (respectively).

For the sake of this presentation, we will only be considering *determinisitic* profilers, although there are 3rd-party libraries out there for *statistical* profiling, such as [`statprof`](https://pypi.python.org/pypi/statprof/0.1.2) and [`pprofile`](https://github.com/vpelletier/pprofile)

## Profiling allows us to measure resources used by sections of the program. 

Typical resources of interest are
* Amount of CPU used  **&larr; our main objective**
* Amount of RAM used
* Disk I/O
* Network I/O
* ...

# The Golden Rule of Programming

“First make it work. Then make it right. Then make it fast.” -Kent Beck

Start simple, switch to more complex techniques if needed!<br><br>
If anybody has every tried to optimize their code, they have probably heard the phrase "Optimize LAST!" The reason this is said is because optimizing is expensive, with many degrees of diminishing returns.

**Profiling** allows one to get the most out of your optimizations.

Which, inherently, means that you will *not* be optimizing everything.

Techniques for measuring CPU time:
1. Manual timeing
2. `timeit` module
3. `profile` or `cprofile` modules

## Case study: filling a grid with point values

* Consider a rectangular 2D grid

<center>
<img align="center" src="https://elearning-gilman.remote-learner.net/pluginfile.php/80970/mod_resource/content/1/grid.png" width="150" height="150">
</center>
* A NumPy array `a[i,j]` holds values at the grid points

# An implementation

In [ ]:
import numpy as np

class Grid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = np.arange(xmin, xmax+dx, step=dx)
        self.ycoor = np.arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        lx = size(self.xcoor)
        ly = size(self.ycoor)
        a = np. zeros((lx,ly))

        for i in range(lx):
            x = self.xcoor[i]
            for j in range(ly):
                y = self.ycoor[j]
                a[i,j] = f(x, y)
        return a

# Usage

Create a new grid:

In [ ]:
g = Grid2D(dx=0.001, dy=0.001)

Computing grid values:

In [ ]:
import numpy as np

def myfunc(x, y):
    return np.sin(x*y) + y

print("Computing values...")
a = g.gridloop(myfunc)
print("done")

Getting grid values at a specific point:

In [ ]:
i=1000
j=1000
print(f'value at ({g.xcoor[i]}, {g.ycoor[j]}) is {a[i,j}')

# Timing

Use `time.time()` to measure the time spend in a code section.
  ```python
  t0 = time.time()
  # execute code here
  t1 = time.time()
  print(f'Runtime: {t1-t0}')
  ```
  

* Simple statements should be placed in a loop.
* Make sure to use a constant machine load.
* Run the tests several times, choose the fastest.

*Note*: `time.time` measures wall clock time. Use `time.clock()` to measure CPU time.

## Timing of the case study

The case study has two parts that could potentially be slow: 
1. The initialisation `Grid2D(dx=0.001, dy=0.001)`
2. Calling the `g.gridloop(myfunc)` function.

We time these two parts separately to figure out how much time is spend in each.

### Timing the Grid2D initialization

In [ ]:
import time

for i in range(1, 4):
    t0 = time.time()
    g = Grid2D(dx=0.001, dy=0.001)
    t1 = time.time()
    print(f'Experiment {i}, CPU time: {t1-t0:.4} s')
print("Done")

### Timing the `gridloop` function

In [ ]:
import time

for i in range(1, 4):
    t0 = time.time()
    g.gridloop(myfunc)
    t1 = time.time()
    print(f'Experiment {i}. CPU time: {t1-t0:.4} s')
print("Done")

$=>$ The gridloop function is the cause of the slow execution!

# The *timeit* module

## The *timeit* module (1)

The `timeit` module provides an convienient way for measuring the CPU time of small code snippets.

Usage:

In [ ]:
import timeit

timeit.timeit(stmt="a+=1",setup="a=0")

Adding the `number` keyword executes the statement multiple times and returns the **accumulated** runtime:

In [ ]:
timeit.timeit(stmt="a+=1",setup="a=0", number=10000)

Use `timeit.repeat` if you would like to repeat the experiment multiple times:

In [ ]:
timeit.repeat(stmt="a+=1",setup="a=0", number=10000, repeat=5)

## The *timeit* module (2)

* Automatically wraps the code in a for - loop.

* Users can provide their own timer (callback):

In [ ]:
timeit.timeit(stmt="sleep(1)", setup="from time import sleep", number=1, timer=time.time)  # Wall-clock time

In [ ]:
timeit.timeit(stmt="sleep(1)", setup="from time import sleep", number=1, timer=time.clock)  # CPU time

* Isolates the global namespace.

* Time a user defined function:

In [ ]:
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

## Using the `cProfile` module

* As a script: `python -m cProfile script.py`

* As a module:

In [ ]:
import cProfile

pr = cProfile.Profile()
res = pr.run("g.gridloop(myfunc)")
pr.dump_stats("gridloop.prof")  # Dump statistics to file for use with pstats

## CAVEAT: Calibrate `profile` (not `cProfile`) to avoid measuring CPU overhead
### since the `profile` module is pure Python based

The `profile` module needs to compensate for CPU overhead caused by internal Python function calls:

   1. Find the profiling overhead: 
     ```python
     overhead = pr.calibrate(number)
     ```
   2. Remove profiling overhead:
      ```python
      pr = profile.Profile(bias=overhead)
      ```

## Profile a function call while keeping its return value

```python
pr = cProfile.Profile()
retval = pr.runcall(func, *args, **kwargs)
```

## Getting runtime statistics

* We can view some runtime statistics with

In [ ]:
res.print_stats()

## Statistics header definitions
* **ncalls**: number of calls
* **tottime**: total time spent in the given function excluding time made in calls to sub-functions
* **percall**: tottime divided by ncalls
* **cumtime**: cumulative time spent in this and all subfunctions
* **percall**: cumtime divided by ncalls
* **filename:lineno(function)**: information about the function

## The `pstats` module

There are many ways to view profiling data.

The module `pstats` provides the class `Stats` for creating profiling reports:

In [ ]:
import pstats

stats = pstats.Stats("gridloop.prof")
stats.sort_stats("time")
stats.print_stats(3)

* The method `sort_stats(key, *keys)` is used to sort future output.
* Common used keys: `'calls', 'cumulative', 'time'`.
* The argument to `print_stats` limits the view to the N most significant results.

## Back to the array:

The total runtime (with `profile`) is 8.1s. Biggest contributors:
   1. `gridloop` contributes ca 41% of the total runtime.
   2. `myfunc` calls contributes ca 55% of the total runtime.

* `myfunc` is fairly straight-forward
  ```python
  def myfunc(x, y):
    return sin(x*y) + y
  ```
  Might be difficult to improve.
* What about `gridloop`?

## Recall that, `gridloop` was a function of the form

```python
def gridloop(self, f):
    lx = size(self.xcoor)
    ly = size(self.ycoor)
    a = np.zeros((lx,ly))

    for i in range(lx):
        x = self.xcoor[i]
        for j in range(ly):
            y = self.ycoor[j]
             a[i,j] = f(x, y)
    return a
```

It would be useful to see how much time is spend in each line!

## Line by line profiling

The `line_profiler` inspects the time spend in each line of a Python function. 

## Usage

1. Install with `pip install line_profiler`
2. "Decorate" the function of interest with `@profile`:
    ```python
    @profile
    def gridloop(func):
        # ...
    ```
3. Run line profiler with:
    ```bash
    kernprof -l -v grid2d_lineprofile.py
    ```

## Demo

In [ ]:
!kernprof -l -v grid2d_lineprofile.py

**Conclusion:** Most of the time is spend in loops and indexing.

## A vectorized Grid2D implementation

In [ ]:
class VectorizedGrid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = np.arange(xmin, xmax+dx, step=dx)
        self.ycoor = np.arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        return f(self.xcoor[:,None], self.ycoor[None,:])  # Vectorized grid evaluation

## Timing the vectorized Grid2D

In [ ]:
g = Grid2D(dx=0.001, dy=0.001)
print("Original Grid")
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

In [ ]:
vg = VectorisedGrid2D(dx=0.001, dy=0.001)
print("Vectorized Grid")
timeit.repeat(stmt="vg.gridloop(myfunc)", setup="from __main__  import vg, myfunc", repeat=5, number=1)

**Vectorization yields a 70 times speed improvement!**

## Visual profiling

A big problem in all of the above techniques is that for a large code base, such a profile can be extremely painful to comb through.

Therefore, a GUI approach has been designed. Many packages are out there, but the one most supported by Continuum Analytics is `snakeviz`

`snakeviz` digests the profile and shows you a visual representation and interactive sunburst graph to show you which functions are taking up the most time/resources

### Implementation

In [ ]:
!pip install snakeviz

In [ ]:
%load_ext snakeviz

g = Grid2D(dx=0.001, dy=0.001)

In [ ]:
%%snakeviz

g.gridloop(myfunc)

There is also a CLI tool to access these same results outside of a jupyter notebook:

```bash
python -m cProfile -o my_program.prof my_program.py
snakeviz program.prof
```

## Lastly, profiler decorating

```python
import cProfile

def profileit(func):
    def wrapper(*args, **kwargs):
        datafn = func.__name__ + ".prof" # Name the data file sensibly
        prof = cProfile.Profile()
        retval = prof.runcall(func, *args, **kwargs)
        prof.dump_stats(datafn)
        return retval

    return wrapper

@profileit # This decorator now will profile the specific function it wraps
def myFunc()
```